In [ ]:
import numpy as np
from scipy.optimize import linprog

def simplex_method(c, A, b):
    """Solve LP: Maximize c^T x subject to Ax <= b, x >= 0"""
    m, n = A.shape
    
    # Create tableau with slack variables
    tableau = np.zeros((m + 1, n + m + 1))
    tableau[:-1, :n] = A
    tableau[:-1, n:n+m] = np.eye(m)
    tableau[:-1, -1] = b
    tableau[-1, :n] = -c
    
    # Variable names and basic variables
    var_names = [f'x{i+1}' for i in range(n)] + [f's{i+1}' for i in range(m)]
    basic = list(range(n, n + m))
    
    iteration = 0
    
    while True:
        iteration += 1
        print(f"\n{'='*60}\nITERATION {iteration}\n{'='*60}\n")
        
        # Print tableau
        print("Current Tableau:")
        print(f"{'Var':<8}" + ''.join(f'{var:>10}' for var in var_names) + f"{'RHS':>10}")
        
        for i in range(m):
            row_var = var_names[basic[i]]
            print(f"{row_var:<8}" + ''.join(f'{tableau[i,j]:>10.2f}' for j in range(n+m)) + 
                  f"{tableau[i,-1]:>10.2f}")
        
        print(f"{'Z':<8}" + ''.join(f'{tableau[-1,j]:>10.2f}' for j in range(n+m)) + 
              f"{tableau[-1,-1]:>10.2f}")
        
        # Find entering variable
        pivot_col = np.argmin(tableau[-1, :-1])
        if tableau[-1, pivot_col] >= 0:
            break
        
        # Find leaving variable
        ratios = [(tableau[i, -1] / tableau[i, pivot_col], i) 
                  for i in range(m) if tableau[i, pivot_col] > 0]
        
        if not ratios:
            return None, None
        
        _, pivot_row = min(ratios)
        
        print(f"\nEntering variable: {var_names[pivot_col]}")
        print(f"Outgoing variable: {var_names[basic[pivot_row]]}")
        
        # Pivot
        tableau[pivot_row] /= tableau[pivot_row, pivot_col]
        for i in range(m + 1):
            if i != pivot_row:
                tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]
        
        basic[pivot_row] = pivot_col
    
    # Extract solution
    solution = np.zeros(n)
    for i, var_idx in enumerate(basic):
        if var_idx < n:
            solution[var_idx] = tableau[i, -1]
    
    print(f"\n{'='*60}\nOPTIMAL SOLUTION FOUND\n{'='*60}")
    print(f"\nCoal A (x1): {solution[0]:.0f} tons")
    print(f"Coal B (x2): {solution[1]:.0f} tons")
    print(f"Coal C (x3): {solution[2]:.0f} tons")
    print(f"Maximum Profit: {tableau[-1, -1]:.0f} BDT")
    
    return solution, tableau[-1, -1]

# Problem setup
c = np.array([12, 15, 14])  # Profit coefficients
A = np.array([[1, 1, 1],     # Total ≤ 100
              [0.02, 0.04, 0.03],  # Phosphorous ≤ 3
              [3, 2, 5]])    # Ash ≤ 300
b = np.array([100, 3, 300])

print("COAL BLENDING PROBLEM - SIMPLEX METHOD") 
x, obj = simplex_method(c, A, b)

# Verify with scipy
print(f"\n{'='*60}\nVERIFICATION WITH SCIPY")
result = linprog(c=-c, A_ub=A, b_ub=b, bounds=(0, None), method='highs')
print(f"Coal A: {result.x[0]:.0f} tons, Coal B: {result.x[1]:.0f} tons, Coal C: {result.x[2]:.0f} tons")
print(f"Maximum Profit: {-result.fun:.0f} BDT")

COAL BLENDING PROBLEM - SIMPLEX METHOD

ITERATION 1

Current Tableau:
Var             x1        x2        x3        s1        s2        s3       RHS
s1            1.00      1.00      1.00      1.00      0.00      0.00    100.00
s2            0.02      0.04      0.03      0.00      1.00      0.00      3.00
s3            3.00      2.00      5.00      0.00      0.00      1.00    300.00
Z           -12.00    -15.00    -14.00      0.00      0.00      0.00      0.00
[(100.0, 0), (75.0, 1), (150.0, 2)]

Entering variable: x2
Outgoing variable: s2

ITERATION 2

Current Tableau:
Var             x1        x2        x3        s1        s2        s3       RHS
s1            0.50      0.00      0.25      1.00    -25.00      0.00     25.00
x2            0.50      1.00      0.75      0.00     25.00      0.00     75.00
s3            2.00      0.00      3.50      0.00    -50.00      1.00    150.00
Z            -4.50      0.00     -2.75      0.00    375.00      0.00   1125.00
[(50.0, 0), (150.0, 1), (75.